# Fitting Decision Models

In [7]:
!pip install -q otter-grader

import otter
grader = otter.Notebook("hw8.ipynb")

import numpy as np
import itertools

When we previously assessed categorization models, we evaluated their ability to predict the empirical proportion of participants that assign a new stimulus to category A versus category B. As we've mentioned before, the same can be done for decision models, where we analogously predict the proportion of participants choosing gamble A over gamble B.

Below are three familiar choice problems that we've seen before. Since pandas dataframes aren't as well suited to this kind of data, we provide each choice problem as a separate Python dictionary within a Python list. For each, we can see the relevant outcomes and probabilities. Additionally, we now also include `prob_chose_A`, which is the empirical proportion of participants choosing gamble A over gamble B.

In [3]:
data = [
    {
        "A_outcomes": np.array([1000., 0.]),
        "B_outcomes": np.array([500.]),
        "A_probs": np.array([0.5, 0.5]), 
        "B_probs": np.array([1.0]),
        "prob_chose_A": 0.08,
    },
    {
        "A_outcomes": np.array([-100., 100.]),
        "B_outcomes": np.array([0.]),
        "A_probs": np.array([0.5, 0.5]),
        "B_probs": np.array([1.0]),
        "prob_chose_A": 0.33,
    },
    {
        "A_outcomes": np.array([500., 0.]),
        "B_outcomes": np.array([5.]),
        "A_probs": np.array([0.01, 0.99]),
        "B_probs": np.array([1.0]),
        "prob_chose_A": 0.60,
    }
]

As previously discussed, we can use decision models to predict these proportions by converting gamble values to probabilities using the Luce choice rule:
$$\frac{e^{ V(A)}}{e^{V(A)} + e^{V(B)}},$$

where $V(A)$ is the value of gamble A and $V(B)$ is the value of gamble B.

This rule is identical to the similarity-choice rule used in our categorization models, but instead of taking similarity values as input, it takes values of gambles as input. Thus, just like our categorization models, we can include a parameter analogous to the sensitivity parameter. In the context of decision models, this model is often called the temperature parameter $\large{\tau}$. The modified Luce choice rule then becomes:

$$\frac{e^{ V(A) \large{\tau}}}{e^{V(A) \large{\tau}} + e^{V(B) \large{\tau}}}.$$

Note that $\large{\tau}$ is still a "sensitivity" parameter and thus functions the same. That is, lower values of $\large{\tau}$ lead to less differentiated gamble values and probabilities closer to 0.5, and higher values of $\large{\tau}$ lead to more differentiated gamble values and more exaggerated probabilities that are further from 0.5.

**Assignment:** Find the parameters ($\alpha$, $\lambda$, $\gamma$, and $\large{\tau}$) of the decision model that best predicts human response proportions in the choice problems in `data`. Use MSE to score each combination of the parameters. Store the best parameters in variables called `best_alpha`, `best_lambda`, `best_gamma`, and `best_tau`. And store the best MSE score in `best_score`.

**Note 1:** The specific parameter values to evaluate are provided below (e.g., `alphas_to_test`).

**Note 2:** You will need to reference the "Models 1", "Models 2", "Decisions 1" and "Decisions 2" labs.

In [10]:
# do not change
alphas_to_test = np.arange(0.01, 1.01, 0.01)
lambdas_to_test = np.linspace(0.0, 3.5, 36)
gammas_to_test = np.arange(0.1, 1.1, 0.1)
taus_to_test = np.arange(0.01, 0.11, 0.01)

# Your code here
def loss_aversion(x, alpha, lam):
    return np.power(x, alpha) if x >= 0 else -lam * np.power(-x, alpha)

def pi(p, gamma):
    return (p ** gamma) / np.power(p ** gamma + (1 - p) ** gamma, 1 / gamma)

def prospect_theory(outcomes, probs, alpha, lam, gamma):
    v  = np.array([loss_aversion(x, alpha, lam) for x in outcomes])
    w  = np.array([pi(p, gamma) for p in probs])
    return np.sum(w * v)

best_score  = np.inf
best_alpha  = None
best_lambda = None
best_gamma  = None
best_tau    = None

for alpha, lam, gamma, tau in itertools.product(alphas_to_test, lambdas_to_test, gammas_to_test, taus_to_test):

    sq_errs = []
    for d in data:
        V_A = prospect_theory(d["A_outcomes"], d["A_probs"], alpha, lam, gamma)
        V_B = prospect_theory(d["B_outcomes"], d["B_probs"], alpha, lam, gamma)
        
        P_A = np.exp(tau * V_A) / (np.exp(tau * V_A) + np.exp(tau * V_B))

        sq_errs.append((P_A - d["prob_chose_A"])**2)

    mse = np.mean(sq_errs)

    if mse < best_score:
        best_score  = mse
        best_alpha  = alpha
        best_lambda = lam
        best_gamma  = gamma
        best_tau    = tau

In [11]:
grader.check("q1")

q1 results: All test cases passed!